## Prepare train and test

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(1, '../src')
from src import PROJECT_PATHS

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.pipeline import Pipeline

In [5]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
df = pd.read_csv(f"{PROJECT_PATHS.data}\\processed\\final_embeded.csv", index_col=0)
df.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_text_0,...,emb_title_290,emb_title_291,emb_title_292,emb_title_293,emb_title_294,emb_title_295,emb_title_296,emb_title_297,emb_title_298,emb_title_299
0,0.0,50,7,-5.000000e-01,8.660254e-01,0.5,0.866025,-0.222521,-0.974928,0.016769,...,-0.006696,0.007845,-0.022188,-0.030541,0.033095,0.005611,0.079799,-0.060192,0.025157,-0.014475
1,-39.0,152,7,-1.000000e+00,-1.836970e-16,0.5,0.866025,-0.222521,-0.974928,0.027725,...,0.009607,0.032843,-0.062798,-0.040111,-0.056778,-0.000490,-0.004771,0.044609,0.044650,0.067494
2,186.0,30,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.027246,...,-0.017774,0.024428,-0.010970,-0.017435,-0.016049,-0.002617,0.025836,0.001839,-0.010091,-0.012859
3,48.0,33,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.037644,...,0.003854,0.045091,0.015493,-0.021446,0.014062,-0.003290,0.072985,-0.022510,0.031925,0.009049
4,27.0,57,7,1.224647e-16,-1.000000e+00,0.5,0.866025,-0.222521,-0.974928,0.033379,...,-0.028516,0.010097,-0.012332,-0.029570,0.022079,-0.014889,0.041341,-0.025642,-0.012887,-0.030432


In [7]:
X, y = df.drop(columns=['target']), df['target']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [9]:
len(X_test)

23895

## Classification

### SVC

In [21]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', SVC())
    ]
)

In [22]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

In [23]:
pipe.score(X_test, y_test)

0.6847876124712283

### LogisticRegression

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', LogisticRegression())
    ]
)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

c:\users\stanislav\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6848294622305922

### LDA

In [19]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [20]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', LDA())
    ]
)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.6759991630048128

## TopicModeling

### LDA

Remove the stop words from the texts, apply lemmatization. After that, let's look at the top 10 words in each topic modeled by the LDA algorithm.

In [80]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
sw = stopwords.words("russian")

def remove_stop_words(words_list):
    return [x for x in words_list if x not in sw]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stanislav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stanislav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [81]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(f"{PROJECT_PATHS.data}\\processed\\preprocessed.csv", index_col=0)

In [82]:
df_oh = df.loc[:, ['TEXT', 'target']]

In [94]:
from pymystem3 import Mystem

mystem = Mystem() 

#df_oh.loc[:, 'cleared_text'] = df_oh['TEXT'].map(lambda x: re.sub('\d+', '0', str(x)))
df_oh['TEXT'] = df_oh['TEXT'].apply(lambda x: " ".join(mystem.lemmatize(word) for word in str(x).split(" ")))
df_oh['TEXT'].tolist()

TypeError: sequence item 0: expected str instance, list found

In [21]:
df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(remove_stop_words(x.split())))

<ipython-input-21-9f0d706d396d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(remove_stop_words(x.split())))


In [41]:
df_oh['TEXT'] = df_oh['TEXT'].apply(lambda x: re.sub('\d+', '0', x))


vectorizer = TfidfVectorizer()
df_oh.head()

<ipython-input-41-056f4790a99f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oh['TEXT'] = df_oh['TEXT'].apply(lambda x: re.sub('\d+', '0', x))


,TEXT,target
0,мвд россии разработало порядок доставления нет...,7
1,хотите национализм шовинизм числе разжигание н...,7
2,депутаты томаринского городского округа юг сах...,7
3,глава «роскосмоса» дмитрий рогозин встрече пре...,7
4,москва 0 февраля тасс рф ждет реакцию великобр...,7


In [23]:
X_tf_idf = vectorizer.fit_transform(df_oh['TEXT'].tolist())

In [24]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=8, random_state=1)
lda.fit(X_tf_idf)

LatentDirichletAllocation(n_components=8, random_state=1)

In [26]:
vocab = vectorizer.get_feature_names()

n_top_words = 10

topic_words = {}

for topic, comp in enumerate(lda.components_):    
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    print([vocab[x] for x in word_idx],"\n")

['аннель', 'ендрит', 'арвен', 'scp', 'протоклетка', 'семенихина', 'радуй', 'ивгур', 'поильник', 'негритёнка'] 

['баюн', 'ладислав', 'хьялдур', 'ермолай', 'шаройко', 'рукоблудием', 'пцу', 'буханкин', 'снорри', 'хамус'] 

['кисё', 'у0', 'басё', 'йес', 'въ', 'анкетыhttps', 'spreadsheet', 'h0jp0b0i', 'анкетамиhttps', 'fkjg'] 

['артес', 'гераськин', 'полуярок', 'марвин', 'morant', 'кобр', 'мересанк', 'вано', 'иша', 'пико'] 

['нe', 'нa', 'чтo', 'кaк', 'oн', 'тo', 'этo', 'нo', 'пo', 'тaк'] 

['украины', 'россии', 'сша', 'http', 'ru', 'украине', 'также', 'заявил', 'рф', 'власти'] 

['это', 'всё', 'очень', 'просто', 'лет', 'время', 'ещё', 'день', 'её', 'нам'] 

['kota', 'stranitsa', 'агидель', 'uchenogo', 'тутси', 'лп', 'зубенко', 'пашаев', 'covideo', 'катри'] 



## Clustering

### Kmeans-fasttext

In [26]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn import metrics

In [27]:
kmeans = KMeans(n_clusters=8).fit(X)

In [28]:
kmeans.predict(X)

array([3, 1, 1, ..., 1, 3, 3], dtype=int32)

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder().fit(y)

In [30]:
np.unique(kmeans.predict(X))

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [31]:
Counter(kmeans.predict(X)).most_common()

[(1, 44800),
 (3, 2920),
 (5, 1453),
 (0, 1008),
 (6, 638),
 (4, 404),
 (2, 158),
 (7, 29)]

In [32]:
Counter(y).most_common()

[(4, 13169),
 (3, 9441),
 (7, 9273),
 (1, 4864),
 (5, 4763),
 (6, 4357),
 (0, 3262),
 (2, 2281)]

In [33]:
le.transform(y)

array([0, 6, 7, ..., 7, 1, 7])

In [34]:
metrics.adjusted_rand_score(kmeans.predict(X), le.transform(y))

0.013200719914041453

### Kmeans-tfifd

In [35]:
km_tfidf=KMeans(n_clusters=8).fit(X_tf_idf)

In [36]:
X_tf_idf.shape

(51416, 151580)

In [37]:
metrics.adjusted_rand_score(km_tfidf.predict(X_tf_idf), le.transform(df_oh['target']))

0.16233893715582967

### OPTICS

In [ ]:
from sklearn.cluster import OPTICS

opt = OPTICS().fit(X)